# LangChain: Q&A over Documents
One of the complex system is to develop a syem whcih can answer questions on top of about the given document. Example: from PDF, word.

This is powerful, because it starts to combine the LLM with the data whcih it was not trained earlier. So much more flexible and adptable to used case.  This is also exciting becasue we'll start to ove beyond langauge models, prompts, and output parsers and start introducing some more of the ley components of LangChain, such as __embedding models__ and __vector stores__.

The general idea here si to use language model and combine it with the docuemnts we have. But there is a key issue.
- LLM's can only inspect a few thousands words at a time

So if we have very large docuemnt how can we use langague model answer qiestion that is there. This is where __embedding__ and __vector storage__ come to play.

## Embedding
- Embedding creates numerical representaions of text. this numerical represenation captures the semantic meaning of the text.
- Embedding vectors captures content/meaning
- text with similar content will ahve similar vecors.

<center><img src='embeddings.PNG' width=250px></center> 


## Vector Database
- A vector database is where it stores vector representaion that we created in embedding
- we break the big documnt in the chunks so that we can only pass the most importatn things in the LLM. 
- We create index and we can use it over runtime and find the pieces of text that is more relevant.

<center><img src='vectorDatabase.PNG' width=250px></center> 


---
An example might be a tool that would allow you to query a product catalog for items of interest.

In [22]:
#pip install --upgrade langchain

In [23]:
# import environemnt variables.
import os

from dotenv import load_dotenv, find_dotenv
_ = load_dotenv(find_dotenv()) # read local .env file

Note: LLM's do not always produce the same results. When executing the code in your notebook, you may get slightly different answers that those in the video.

In [24]:
# account for deprecation of LLM model
import datetime
# Get the current date
current_date = datetime.datetime.now().date()

# Define the date after which the model should be set to "gpt-3.5-turbo"
target_date = datetime.date(2024, 6, 12)

# Set the model variable based on the current date
if current_date > target_date:
    llm_model = "gpt-3.5-turbo"
else:
    llm_model = "gpt-3.5-turbo-0301"

In [25]:
from langchain.chains import RetrievalQA # retrival over some documeopts 
from langchain.chat_models import ChatOpenAI # import LLM model 
from langchain.document_loaders import CSVLoader # import document laoder
from langchain.vectorstores import DocArrayInMemorySearch # import vector store. this is in memory vector store and doesnt need external
from IPython.display import display, Markdown # markdown for visualization in jupyter 
from langchain.llms import OpenAI

In [26]:
# CSV loader with the path
file = 'OutdoorClothingCatalog_1000.csv'
loader = CSVLoader(file_path=file)

In [27]:
# import index. `VectorstoreIndexCreator`. It help us create vector store easily.
from langchain.indexes import VectorstoreIndexCreator

In [28]:
#pip install docarray

In [29]:
# create index
index = VectorstoreIndexCreator(
    vectorstore_cls=DocArrayInMemorySearch # define vector store clss
).from_loaders([loader]) # takes list of documents

In [30]:
# now its created and we can start question.
query ="Please list all your shirts with sun protection \
in a table in markdown and summarize each one."

**Note**:
- The notebook uses `langchain==0.0.179` and `openai==0.27.7`
- For these library versions, `VectorstoreIndexCreator` uses `text-davinci-003` as the base model, which has been deprecated since 1 January 2024.
- The replacement model, `gpt-3.5-turbo-instruct` will be used instead for the `query`.
- The `response` format might be different than the video because of this replacement model.

In [31]:
llm_replacement_model = OpenAI(temperature=0, 
                               model='gpt-3.5-turbo-instruct')

# create response and ask query
response = index.query(query, 
                       llm = llm_replacement_model)

In [32]:
display(Markdown(response))



| Name | Description | Sun Protection Rating |
| --- | --- | --- |
| Men's Tropical Plaid Short-Sleeve Shirt | Made of 100% polyester, UPF 50+ rating, wrinkle-resistant, front and back cape venting, two front bellows pockets | SPF 50+, blocks 98% of harmful UV rays |
| Men's Plaid Tropic Shirt, Short-Sleeve | Made of 52% polyester and 48% nylon, UPF 50+ rating, SunSmart technology, wrinkle-free, front and back cape venting, two front bellows pockets | SPF 50+, blocks 98% of harmful UV rays |
| Men's TropicVibe Shirt, Short-Sleeve | Made of 71% nylon and 29% polyester, UPF 50+ rating, front and back cape venting, two front bellows pockets | SPF 50+, blocks 98% of harmful UV rays |
| Sun Shield Shirt | Made of 78% nylon and 22% Lycra Xtra Life fiber, UPF 50+ rating, moisture-wicking, abrasion-resistant, fits over swimsuit | SPF 50+, blocks 98% of harmful UV rays |

## Step By Step

In [33]:
# create a document loader that is created from the CSV and this is where
# we want to gdo all the questions and answer from the docuemtn about the review

from langchain.document_loaders import CSVLoader
loader = CSVLoader(file_path=file)

In [34]:
# load docuemtns from docuemnt laoder. 
docs = loader.load()

In [35]:
#  Each docuemnt coresponds to an individual product.
# as these docuemtns are small so we dont tneed to do any chunking. So we can create embedding directly.
docs[0]

Document(page_content=": 0\nname: Women's Campside Oxfords\ndescription: This ultracomfortable lace-to-toe Oxford boasts a super-soft canvas, thick cushioning, and quality construction for a broken-in feel from the first time you put them on. \n\nSize & Fit: Order regular shoe size. For half sizes not offered, order up to next whole size. \n\nSpecs: Approx. weight: 1 lb.1 oz. per pair. \n\nConstruction: Soft canvas material for a broken-in feel and look. Comfortable EVA innersole with Cleansport NXT® antimicrobial odor control. Vintage hunt, fish and camping motif on innersole. Moderate arch contour of innersole. EVA foam midsole for cushioning and support. Chain-tread-inspired molded rubber outsole with modified chain-tread pattern. Imported. \n\nQuestions? Please contact us for any inquiries.", metadata={'source': 'OutdoorClothingCatalog_1000.csv', 'row': 0})

In [36]:
# Create embedding/ import and intialise
from langchain.embeddings import OpenAIEmbeddings
embeddings = OpenAIEmbeddings()

In [37]:
# to take a look of what his embeding do. What happen when we abmbed aprticaular piece text
embed = embeddings.embed_query("Hi my name is Abhishek")

In [38]:
print(len(embed))
# We can see there are over 1000 differnet elemetns. Each of these elaments have a diffentrent numerical value.

1536


In [39]:
print(embed[:5])

[-0.01572434790432453, -0.003083205549046397, -0.016520842909812927, -0.02434447780251503, -0.03597073256969452]


In [40]:
# create embeddign for al the documents we uploaded and store in vector store.
db = DocArrayInMemorySearch.from_documents(
    docs, 
    embeddings
)

In [41]:
# we can use the vector store to find pieces of text simialr to our query
query = "Please suggest a shirt with sunblocking"

In [43]:
# do similarity search method in the vectror storage and pass a query. we will get a list of documtns
docs = db.similarity_search(query)

In [44]:
len(docs)
# return 4 documets

4

In [45]:
docs[0]
# 

Document(page_content=': 255\nname: Sun Shield Shirt by\ndescription: "Block the sun, not the fun – our high-performance sun shirt is guaranteed to protect from harmful UV rays. \n\nSize & Fit: Slightly Fitted: Softly shapes the body. Falls at hip.\n\nFabric & Care: 78% nylon, 22% Lycra Xtra Life fiber. UPF 50+ rated – the highest rated sun protection possible. Handwash, line dry.\n\nAdditional Features: Wicks moisture for quick-drying comfort. Fits comfortably over your favorite swimsuit. Abrasion resistant for season after season of wear. Imported.\n\nSun Protection That Won\'t Wear Off\nOur high-performance fabric provides SPF 50+ sun protection, blocking 98% of the sun\'s harmful rays. This fabric is recommended by The Skin Cancer Foundation as an effective UV protectant.', metadata={'source': 'OutdoorClothingCatalog_1000.csv', 'row': 255})

# how can we utilize that for question answering?

In [46]:
# Create retriever: retriever is a genric interface that can be 
# underpinned by any methods that takes query and return documetns. Embedding and vector storage is one method to do so. 
retriever = db.as_retriever()

In [47]:
# because we want to do text generation so well use langage model. Use caht openAI
llm = ChatOpenAI(temperature = 0.0, model=llm_model)

In [48]:
# If we wanna do it bu hands , we will combine all the docmunets into single text.
# we join all the page contents in a varaible
qdocs = "".join([docs[i].page_content for i in range(len(docs))])


In [49]:
# pass that varaible ans ask query
response = llm.call_as_llm(f"{qdocs} Question: Please list all your \
shirts with sun protection in a table in markdown and summarize each one.") 


In [57]:
display(Markdown(response))

| Shirt Name                           | Description                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                

## Stuff Method
stuffing is the simplest metod. You simply stuff all data into the prompt as context to pass to the langauge model

Pros: It makes a single call to the LLM. The LLM has access to all the data at once.


Cons: LLMs have a contect length, and for large documents or many documents this will not work as it will result in a prompt larger than the conetext length.

In [51]:
# All the text must be encapsulated with langchain chain
qa_stuff = RetrievalQA.from_chain_type( # create a retrival QA cahin
    llm=llm,  
    chain_type="stuff",  # simplest method that stuff all in one text.
    retriever=retriever,  # pass retriever for fetching documnts and pass to langague model 
    verbose=True
)

In [52]:
query =  "Please list all your shirts with sun protection in a table \
in markdown and summarize each one."

In [58]:
# run the cahin on the above query
response = qa_stuff.run(query)



> Entering new RetrievalQA chain...

> Finished chain.


In [59]:
display(Markdown(response))

| Shirt Name                           | Description                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                

In [55]:
response = index.query(query, llm=llm)

In [56]:
# look it in more detailed in a customized form
index = VectorstoreIndexCreator(
    vectorstore_cls=DocArrayInMemorySearch,
    embedding=embeddings,
).from_loaders([loader])

## Additional methods

1.map_reduce: This takes all the chunks passes them along with the questions to the languge meode and than use another language model to get final answer. Treat individual documents
2. Refine: depend n previous docuemnts
3. Map_rerank

<center><img src='methods.PNG' width=250px></center> 